In [16]:
import matplotlib.pyplot as plt
import vtk
import numpy as np
import sys
import math
import os
import glob
from vtk.util.numpy_support import *
import pandas
from multiprocessing import Pool

In [33]:
def read_vti(filename):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    return reader.GetOutput()

def write_vti(filename,data):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetInputData(data)
    writer.SetFileName(filename)
    writer.Update()

def compute_3d_to_1d_map(x,y,z,dimx,dimy,dimz):
    return x + dimx*(y+dimy*z)

### Compute 3D SSIM
def compute_3D_SSIM(data1,data2, min_val, max_val):
    if len(data1) != len(data2):
        data2 = data2[0:len(data1)]

    mean1 = np.average(data1)
    mean2 = np.average(data2)
    var1 = np.var(data1)
    var2 = np.var(data2)
    covar = np.abs(np.cov(data1,data2)[0][1])
    
    DR = max_val - min_val
    
    c1 = np.power((0.01*DR),2)
    c2 = np.power((0.03*DR),2)
    ssim = ((2*mean1*mean2 + c1)*(2*covar+c2))/((mean1*mean1 + mean2*mean2 + c1)*(var1*var1+var2*var2+c2))
    return ssim


### Compute 3D SSIM New
def compute_3D_SSIM_new(data1,data2, min_val, max_val):

    # if len(data1) != len(data2):
    #     data2 = data2[0:len(data1)]

    mean1 = np.average(data1)
    mean2 = np.average(data2)
    var1 = np.var(data1)
    var2 = np.var(data2)
    #covar = np.abs(np.cov(data1,data2)[0][1])

    DR = max_val - min_val
    
    c1 = np.power((0.0001*DR),2)
    c2 = np.power((0.0003*DR),2)

    l_comp = (2*mean1*mean2 + c1)/(mean1*mean1 + mean2*mean2 + c1)
    c_comp = (2*var1*var2+c2)/(var1*var1 + var2*var2 + c2)

    if np.min(data1) == 0 and np.max(data1) == 0:
        s_comp = 0.0
    elif np.min(data2) == 0 and np.max(data2) == 0:
        s_comp = 0.0
    else:
        if len(data1) != len(data2):
            data2 = data2[0:len(data1)]
        
        s_comp = np.abs(np.corrcoef(data1,data2)[0][1])

    ssim = l_comp*c_comp*s_comp

    return ssim

def gen_ssim_field_single_tstep(inparam):
    
    ## parse params
    tstep_num = inparam[0]
    out_file_path1 = inparam[1]
    varname = inparam[2]
    final_feature_data = inparam[3]
    xbox = inparam[4]
    ybox = inparam[5]
    zbox = inparam[6]
    totPtsBlock = inparam[7]
 
    ##load data
    data_file = data_path + str(tstep_num) + '.vti'
    data = read_vti(data_file)
    np_var_array = vtk.util.numpy_support.vtk_to_numpy(data.GetPointData().GetArray(varname))
    np_var_array = np.reshape(np_var_array,(zdim,ydim,xdim))
    classified_array = np.zeros_like(np_var_array)
    dims = data.GetDimensions()
    
    # Iterate over each block and compute SSIM
    for k in range(0,dims[2],zbox):
        for j in range(0,dims[1],ybox):
            for i in range(0,dims[0],xbox):
            
                ## note that np_var_array has x and z swapped. SO, k index in given first
                block_data = np_var_array[k:k+zbox, j:j+ybox, i:i+xbox] 
                block_data = block_data.reshape(totPtsBlock)
                
                ssim_val = compute_3D_SSIM(block_data,final_feature_data)
                classified_array[k:k+zbox, j:j+ybox, i:i+xbox] = ssim_val
                
    classified_array = classified_array.reshape(xdim*ydim*zdim)
    classified_array_vtk = vtk.util.numpy_support.numpy_to_vtk(classified_array)
    classified_array_vtk.SetName('feature_similarity')
    data.GetPointData().AddArray(classified_array_vtk)

    out_fname = out_file_path1 + str(tstep_num) + '.vti'
    write_vti(out_fname,data)    

In [8]:
# ## MFIX bubble data
# xdim = 256
# ydim = 64
# zdim = 256

# initstep = 75
# tsteps = 409

# feature_tstep = 190
# feature_data_file = '/disk1/MFIX_bubble_fields_highres/original_timestep_' + str(feature_tstep) + '.vti'

# ##this values come from ParaView now by observing the extract block range

# ## tstep 250
# # xmin = 32
# # xmax = 40
# # ymin = 0
# # ymax = 8
# # zmin = 202
# # zmax = 210

# ## tstep 250
# # xmin = 46
# # xmax = 54
# # ymin = 0
# # ymax = 8
# # zmin = 46
# # zmax = 50

# ## tstep 190
# xmin = 13
# xmax = 17
# ymin = 0
# ymax = 8
# zmin = 220
# zmax = 224

# xbox = xmax - xmin
# ybox = ymax - ymin
# zbox = zmax - zmin

# print xbox, ybox, zbox


# #totPts = xbox*ybox*zbox
# totPtsBlock = xbox*ybox*zbox

# data_path = '/disk1/MFIX_bubble_fields_highres/original_timestep_'
# varname = 'ImageScalars'
# out_file_path1 = '../out/mfix_case_3/ssim_field_'
# out_file_path2 = '../out/mfix_case_3/pervoxel_ssim_field_'

4 8 4


In [38]:
## Vortex  data
xdim = 128
ydim = 128
zdim = 128

initstep = 10
tsteps = 20
window = 10

feature_tstep = 10
feature_data_file = '/disk2/Source_Codes/multivar_sampling/Data/vortex_vti/vortex_' + str(feature_tstep) + '.vti'

##this values come from ParaView now by observing the extract block range

## tstep 10
xmin = 0
xmax = 4
ymin = 68
ymax = 72
zmin = 90
zmax = 94

xbox = xmax - xmin
ybox = ymax - ymin
zbox = zmax - zmin

print xbox, ybox, zbox


#totPts = xbox*ybox*zbox
totPtsBlock = xbox*ybox*zbox

data_path = '/disk2/Source_Codes/multivar_sampling/Data/vortex_vti/vortex_'
varname = 'ImageScalars'
out_file_path1 = '../out/vortex/ssim_field_'
out_file_path2 = '../out/vortex/pervoxel_ssim_field_'


data_max = 10.6 #np.max(feature_data)
data_min = 0 #np.min(feature_data)

print data_max, data_min

4 4 4
10.6 0


In [39]:
## Get the block data which will be treated as the feature of interest
feature_selection_field = read_vti(feature_data_file)

feature_data = vtk.util.numpy_support.vtk_to_numpy(feature_selection_field.GetPointData().GetArray(varname))
feature_data = np.reshape(feature_data,(zdim,ydim,xdim))
feature_data = feature_data[zmin:zmax, ymin:ymax, xmin:xmax]
shape = np.shape(feature_data)
final_feature_data = np.reshape(feature_data,shape[0]*shape[1]*shape[2])

In [40]:
### compare with given feature's SSIM block-wise
for ii in range(initstep,tsteps,window):
   
    data_file = data_path + str(ii) + '.vti'
    
    print data_file
    
    data = read_vti(data_file)
    np_var_array = vtk.util.numpy_support.vtk_to_numpy(data.GetPointData().GetArray(varname))
    ## note that np_var_array has x and z swapped
    
    np_var_array = np.reshape(np_var_array,(zdim,ydim,xdim))
    classified_array = np.zeros_like(np_var_array)
    
    for k in range(0,zdim,zbox):
        for j in range(0,ydim,ybox):
            for i in range(0,xdim,xbox):
            
                ## note that np_var_array has x and z swapped. SO, k index in given first
                block_data = np_var_array[k:k+zbox, j:j+ybox, i:i+xbox] 
                block_data = block_data.reshape(totPtsBlock)
                
                ssim_val = compute_3D_SSIM_new(block_data,final_feature_data, data_min, data_max)
                classified_array[k:k+zbox, j:j+ybox, i:i+xbox] = ssim_val
                
                
    classified_array = classified_array.reshape(xdim*ydim*zdim)
    classified_array_vtk = vtk.util.numpy_support.numpy_to_vtk(classified_array)
    classified_array_vtk.SetName('feature_similarity')
    data.GetPointData().AddArray(classified_array_vtk)

    out_fname = out_file_path1 + str(ii) + '.vti'
    write_vti(out_fname,data)

/disk2/Source_Codes/multivar_sampling/Data/vortex_vti/vortex_10.vti


In [22]:
### Per voxel neighborhood distribution matching instead of block-based comparison

neighborhood_z = 3
neighborhood_y = 3
neighborhood_x = 3

for ii in range(initstep,tsteps):
    
    print 'processing tstep ' + str(ii)
    
    data_file = data_path + str(ii) + '.vti'
    
    data = read_vti(data_file)
    np_var_array = vtk.util.numpy_support.vtk_to_numpy(data.GetPointData().GetArray(varname))
    ## note that np_var_array has x and z swapped
    
    np_var_array = np.reshape(np_var_array,(zdim,ydim,xdim))
    classified_array = np.zeros_like(np_var_array)
    
    ## do for each grid point
    for k in range(0,zdim):
        for j in range(0,ydim):
            for i in range(0,xdim):
            
                ## note that np_var_array has x and z swapped. SO, k index in given first
                
                if (k-neighborhood_z) >= 0:
                    kmin = (k-neighborhood_z)
                else:
                    kmin = 0
                
                if (k+neighborhood_z) <= zdim:
                    kmax = (k+neighborhood_z)
                else:
                    kmax = zdim
                    
                if (j-neighborhood_y) >= 0:
                    jmin = (j-neighborhood_y)
                else:
                    jmin = 0
                
                if (j+neighborhood_y) <= ydim:
                    jmax = (j+neighborhood_y)
                else:
                    jmax = ydim 
                    
                if (i-neighborhood_x) >= 0:
                    imin = (i-neighborhood_x)
                else:
                    imin = 0
                
                if (i+neighborhood_x) <= xdim:
                    imax = (i+neighborhood_x)
                else:
                    imax = xdim    
                
                block_data = np_var_array[kmin:kmax, jmin:jmax, imin:imax] 
                
                totPtsBlock = np.shape(block_data)[0]*np.shape(block_data)[1]*np.shape(block_data)[2]
                
                block_data = block_data.reshape(totPtsBlock)
            
                ssim_val = compute_3D_SSIM(block_data,final_feature_data)
                
                classified_array[k, j, i] = ssim_val
                
                
    classified_array = classified_array.reshape(xdim*ydim*zdim)
    classified_array_vtk = vtk.util.numpy_support.numpy_to_vtk(classified_array)
    classified_array_vtk.SetName('feature_similarity')
    data.GetPointData().AddArray(classified_array_vtk)

    out_fname = out_file_path2 + str(ii) + '.vti'
    write_vti(out_fname,data)

NameError: name 'window' is not defined